# New LiDAR data - Verifying the new sensor

In [ ]:
import open3d as o3d
import cv2
import numpy as np
import os
import copy
import math
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys

from math import sqrt
from collections import defaultdict
from scipy.stats import gaussian_kde

poles_dir = '../../LiDAR/'

So, after the previous LiDAR broke, we got a new LiDAR to take the PCD measurements. However, the behaviour of this device must be verified before it can be used for the field tests. This is a simple notebook to verify that the resolution of the data is nice enough.

And yes, I hope this notebook ends up being simple. Not like the last one. Oh, have mercy on my soul, RNGesus.

## Preliminary analysis

In [ ]:
new_pointcloud_all = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/poleny_1.pts'))
new_pointcloud_pole = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/poleny_2_crop.pts'))
old_pointcloud_all = o3d.io.read_point_cloud(os.path.join(poles_dir, '0/3d/0/cloud_merged.pcd'))
old_pointcloud_pole = o3d.io.read_point_cloud(os.path.join(poles_dir, '0/3d/0/cloud.pcd'))

o3d.visualization.draw([new_pointcloud_pole])

*A priori*, it looks like this new data has less resolution than the previous LiDAR. Checking for the amount of points on each cloud could be a good idea, just to verify.

In [ ]:
print(f'Old pointcloud size (whole scan, merge of 5 frames): {len(old_pointcloud_all.points)} points')
print(f'New pointcloud size (whole scan): {len(new_pointcloud_all.points)} points')
print(f'Old pointcloud size (pole, merge of 5 frames): {len(old_pointcloud_pole.points)} points')
print(f'New pointcloud size (pole): {len(new_pointcloud_pole.points)} points')

As it can be seen, the merged old pointclouds have circa 500000 points, so each frame has approximately 100000 points. However, these points belong to a scan of **a portion of the room**, whereas the new scan has 220000 points corresponding to **a scan of the whole room**.

Furthermore, the portion of the old scan corresponding to **a quarter of the pole** has 38784 points, around 7750 per frame, whereas the new scan with **the whole pole** only has 1032 points. All this evidence points to the fact that this new LiDAR has indeed less resolution than the old one.

Now, even if this new device has too low resolution to accurately characterize the surface of the pole, it has an advantage over the previous one: this one can be moved around more easily, and therefore it can generate **a scan of the whole pole at once**. This greatly simplifies the procedure to check the pole's diameter, since with the other scanner, all different rotations should have been registrated beforehand.

Before giving an estimate of the pole's diameter, it is a good idea to know how the data is represented; i.e., which coordinates represent width, height and depth in this PCD data.

In [ ]:
cropped_pole_points = np.asarray(new_pointcloud_pole.points)
cropped_pole_points = [point for point in cropped_pole_points if point[2] > -0.9]
test_pointcloud = o3d.geometry.PointCloud()
test_pointcloud.points = o3d.utility.Vector3dVector(cropped_pole_points)

o3d.visualization.draw_geometries([test_pointcloud])

By fiddling with this cell, it can be seen that the pole's height is actually represented in the Z coordinate, rather than in the Y coordinate.

## Pole diameter estimation

The following cell implements an algorithm to give an estimate of the pole's diameter that runs in $\mathcal{O}(n^2)$ time w.r.t. the amount of points in the point cloud. For each point, the distance to the point furthest away is computed, but only to the points within a given height threshold w.r.t. the other point (the height difference is corrected by applying Pythagoras). These distances are then employed to give a $\mu \pm \sigma$ estimate of the pole's diameter.

In [ ]:
def get_continuous_mode(estimates):
    distribution = gaussian_kde(estimates)
    x_domain = np.linspace(min(estimates), max(estimates), 1000)
    y_pdf = distribution.pdf(x_domain)
    i = np.argmax(y_pdf)
    return x_domain[i]


def estimate_diameter(pcd, height_threshold):
    estimates = []
    for point in pcd.points:
        points_within_threshold = [p for p in pcd.points if p[2] < point[2]+height_threshold and p[2] > point[2]-height_threshold]
        points_distances = [math.dist(point, p) for p in points_within_threshold]
        idx_max_distance = np.argmax(points_distances)
        max_distance = points_distances[idx_max_distance]
        furthest_point_z_diff = abs(points_within_threshold[idx_max_distance][2] - point[2])
        diameter_estimate = sqrt(max_distance**2 - furthest_point_z_diff**2)
        estimates.append(diameter_estimate)
        
    mu, sigma, mode = np.mean(estimates), np.std(estimates), get_continuous_mode(estimates)
    
    return estimates, mu, sigma, mode

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(test_pointcloud, 0.1)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

According to this algorithm, the estimated diameter of the pole is 22cm (after removing the bottom part, which was altering the estimate). A good way to verify this is to draw a cylinder of this diameter; if it fits inside the PCD, then the estimate should be correct.

In [ ]:
# WARNING! Open3D's "__init__.py" has been edited to remove some Jupyter functionality. Apparently, that sets material attributes as read-only

def verify_algorithm(pcd, mode, sigma):
    # Create and place base cylinder estimate (no +- sigma)
    pcd_points = np.asarray(pcd.points)
    height = np.max(pcd_points[:,2]) - np.min(pcd_points[:,2])
    mesh_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=mode / 2, height=height)
    mesh_center = np.mean(pcd.points, axis=0)
    mesh_cylinder.translate(mesh_center, relative=False)
    
    # Create cylinders for estimates with sigma
    mesh_cylinder_plus_sigma = o3d.geometry.TriangleMesh.create_cylinder(radius=mode / 2 + sigma, height=height)
    mesh_cylinder_minus_sigma = o3d.geometry.TriangleMesh.create_cylinder(radius=mode / 2 - sigma, height=height)
    mesh_cylinder_plus_sigma.translate(mesh_center, relative=False)
    mesh_cylinder_minus_sigma.translate(mesh_center, relative=False)
    
    # Create materials for proper coloring and visualization
    base_material = o3d.visualization.rendering.MaterialRecord()
    base_material.shader = 'defaultLitTransparency'
    base_material.base_color = [1.0, 0.0, 0.0, 0.3]
    
    plus_sigma_material = o3d.visualization.rendering.MaterialRecord()
    plus_sigma_material.shader = 'defaultLitTransparency'
    plus_sigma_material.base_color = [0.0, 1.0, 0.0, 0.3]
    
    minus_sigma_material = o3d.visualization.rendering.MaterialRecord()
    minus_sigma_material.shader = 'defaultLitTransparency'
    minus_sigma_material.base_color = [0.0, 0.0, 1.0, 0.3]
    
    # Link materials to meshes, and draw
    geoms = [
        {'name': 'pcd', 'geometry': pcd},
        {'name': 'base', 'geometry': mesh_cylinder, 'material': base_material},
        {'name': 'plus', 'geometry': mesh_cylinder_plus_sigma, 'material': plus_sigma_material},
        {'name': 'minus', 'geometry': mesh_cylinder_minus_sigma, 'material': minus_sigma_material}
    ]
    
    o3d.visualization.draw(geoms)

In [ ]:
verify_algorithm(test_pointcloud, mode, sigma)

## The remaining poles

This analysis was made only on the scan of pole 41. Now, the scans of all poles are available, and also the measured pole diameters, so having this information, it should be possible to validate this estimation algorithm.

However, the poles must be manually cropped beforehand.

In [ ]:
def crop_pcd(pcd, xbounds, ybounds, zbounds, draw=False):
    cropped_pointcloud_array = []
    pcd_array = np.asarray(pcd.points)
    
    for point in pcd_array:
        if (xbounds[0] <= point[0] <= xbounds[1]) and (ybounds[0] <= point[1] <= ybounds[1]) and (zbounds[0] <= point[2] <= zbounds[1]):
            cropped_pointcloud_array.append(point)
            
    cropped_pointcloud = o3d.geometry.PointCloud()
    cropped_pointcloud.points = o3d.utility.Vector3dVector(cropped_pointcloud_array)
    cropped_pointcloud.estimate_normals()
    
    if draw:
        o3d.visualization.draw_geometries([cropped_pointcloud])
    else:
        return cropped_pointcloud

In [ ]:
new_pole_all = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/poleny_2.pts'))
new_pole_crop = crop_pcd(new_pole_all, [1.05,1.3], [-1.3,-1.05], [-1.05,1])
o3d.io.write_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/poleny_2_crop.pts'), new_pole_crop)

In [ ]:
pole_5_all = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole5_2.pts'))
pole_5_crop = crop_pcd(pole_5_all, [1.38,1.62], [-0.81,-0.59], [-2.05,1])
o3d.io.write_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole5_2_crop.pts'), pole_5_crop)

In [ ]:
pole_6_all = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole6_1.pts'))
pole_6_crop = crop_pcd(pole_6_all, [1.15,1.41], [-0.97,-0.71], [-2.05,1])
o3d.io.write_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole6_1_crop.pts'), pole_6_crop)

In [ ]:
pole_30_all = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole30_2.pts'))
pole_30_crop = crop_pcd(pole_30_all, [0.97,1.25], [-1.45,-1.15], [-2.05,-0.04])
o3d.io.write_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole30_2_crop.pts'), pole_30_crop)

In [ ]:
pole_41_crop = o3d.io.read_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole41_1_crop_old.pts'))
pole_41_crop = crop_pcd(pole_41_crop, [-1.5,-1.23], [-0.8,-0.5], [-10,10])
o3d.io.write_point_cloud(os.path.join(poles_dir, 'NEW_LIDAR/pole41_1_crop.pts'), pole_41_crop)

### Remaining poles - Diameter estimation

All new poles have been manually cropped from the pole scans. Now, the diameter estimation algorithm can be properly validated.

#### New pole

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(new_pole_crop, 0.5)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

#### Pole 5

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(pole_5_crop, 0.5)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

#### Pole 6

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(pole_6_crop, 0.5)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

#### Pole 30

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(pole_30_crop, 0.5)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

#### Pole 41

In [ ]:
estimates, mu, sigma, mode = estimate_diameter(pole_41_crop, 0.5)
plt.hist(estimates)
print(f'Diameter estimation: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')

As it can be seen, this algorithm is relatively precise: all measured diameters fall within a standard deviation of the estimated diameters. However, the estimates for poles 5, 6 and 30 are more below the measurements than for poles 41 and the new pole. I believe this might be due to the irregularity of these poles, as their diameter changes more along their surfaces (or, in the case of pole 30, it has a big crack through which the LiDAR actually detects the inside of the pole).

Therefore, it might be more interesting, following the approach of the X-ray groundtruth, to measure the diameter in equally-sized chunks of the pole.

## Diameter estimate chunkification

In [ ]:
def estimate_diameter_chunkified(pcd, height_threshold, n_chunks, draw=False):
    fig, axes = plt.subplots(ncols=n_chunks, figsize=(5*n_chunks, 5))
    pcd_array = np.asarray(pcd.points)
    min_height, max_height = np.min(pcd_array[:,2]), np.max(pcd_array[:,2])
    chunk_size = (max_height - min_height) / n_chunks
    chunks = [[point for point in pcd_array if ((min_height + chunk_size*i) < point[2] < (min_height + chunk_size*(i+1)))] for i in range(n_chunks)]
    modes = []
    for idx, chunk in enumerate(chunks):
        pcd_chunk = o3d.geometry.PointCloud()
        pcd_chunk.points = o3d.utility.Vector3dVector(chunk)
        estimates, mu, sigma, mode = estimate_diameter(pcd_chunk, height_threshold)
        modes.append(mode)
        axes[idx].hist(estimates)
        print(f'Diameter estimation for chunk {idx}: {mu:.3f} ± {sigma:.3f}, mode: {mode:.3f}')    
        
    if draw:
        geoms = [{'name': 'pcd', 'geometry': pcd}]
        for idx, mode in enumerate(modes):
            mesh_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=mode / 2, height=chunk_size)
            mesh_center = np.mean(pcd_array, axis=0)
            mesh_center[2] = min_height + chunk_size*idx + chunk_size/2
            mesh_cylinder.translate(mesh_center, relative=False)
            chunk_mat = o3d.visualization.rendering.MaterialRecord()
            chunk_mat.shader = 'defaultLitTransparency'
            color = (idx / n_chunks,) * 3
            chunk_mat.base_color = [color[0], color[1], color[2], 0.7]
            geoms.append({'name': f'chunk_{idx}', 'geometry': mesh_cylinder, 'material': chunk_mat})
        o3d.visualization.draw(geoms)

### New pole

In [ ]:
estimate_diameter_chunkified(new_pole_crop, 0.1, 10, True)

### Pole 5

In [ ]:
estimate_diameter_chunkified(pole_5_crop, 0.1, 10, True)

### Pole 6

In [ ]:
estimate_diameter_chunkified(pole_6_crop, 0.1, 10, True)

### Pole 30

In [ ]:
estimate_diameter_chunkified(pole_30_crop, 0.1, 10, True)

### Pole 41

In [ ]:
estimate_diameter_chunkified(pole_41_crop, 0.1, 10, True)

This chunkified analysis provides further insight about the diameter of the poles. This can be seen with a visual scan, but the diameter changes along the vertical axis, and this change can actually give insight about imperfections in the pole surface, such that if the color histograms/Gabor filters detect imperfections, but the diameter estimate is relatively OK, then the imperfection could be any kind of stain.

Furthermore, if an area presents severe surface rot or cracks (such as with poles 5 and 6), the diameter in those areas will be severely affected. This can be seen for the aforementioned poles with a simple visual inspection. For the diameter estimates, it can be seen that, in these areas, the estimates are lower, so this algorithm seems to work properly for diameter estimation.